In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from matplotlib import font_manager, rc
import plotly.express as px
import datetime as dt

In [ ]:
df = pd.read_excel('미니프로젝트-쇼핑몰 실습데이터.xlsx')
df2 = df.copy()
purchase = df2[df2['처리상태'] == '구매확정']     # 구매확정인 값만 출력
purchase = purchase.loc[:, (purchase.columns != '초도상품') & (purchase.columns != '제작문구 내역')]     # 필요없는 변수 제거
purchase = purchase.replace('지니 태블릿(후불집행)','지니 태블릿')
purchase = purchase.reset_index(drop = True)     # 인덱스 초기화

## 3. 결제 방법에 따른 분석
- 결제 방법과 결제 금액의 연관성 분석<br>
(결제 방법은 맨 앞의 한 가지만 사용한 것으로 간주)

In [49]:
# 결제방법에 해당하는 값들 확인
purchase['결제방법'].unique()

array(['신용카드', '가상계좌', '신 + 포', '포인트', '신 + 적 + 포', '신 + 적', '적 + 포',
       '웰컴마일', '신 + 웰컴', '적립금', '후불', '후 + 포', '후 + 적 + 포', nan, '후 + 적',
       '정 + 포', '정기결제', '정 + 적 + 포', '정 + 적', '현금간편결제', '현 + 적', '현 + 포',
       '현 + 적 + 포', '-', ' + 포'], dtype=object)

In [50]:
# 인덱스 초기화 후 새 데이터프레임 생성
option = purchase.reset_index(drop=True)
option.head()

,주문번호,업체명,상품명,제조사,주문수량,판매금액,결제방법,주문일자,처리상태,할부기간,연도,월,업체수
0,191213170236874,지니,eHBM셀파수학교실 비닐봉투-1세트 100장입,HB,1,5000,신용카드,2019-12-13 17:03:37,구매확정,NaN,2019,12,4
1,191216100347429,지니,eHBM셀파수학교실 비닐봉투-1세트 100장입,HB,1,5000,신용카드,2019-12-16 10:04:50,구매확정,NaN,2019,12,4
2,191216100347429,지니,eHBM셀파수학교실 삼색볼펜-1세트 50개입,HB,3,67500,신용카드,2019-12-16 10:04:50,구매확정,NaN,2019,12,4
3,191216173241622,지니,eHBM셀파수학교실 회원용 리플렛-1세트 50권입,HB,2,30000,신용카드,2019-12-16 17:33:43,구매확정,NaN,2019,12,4
4,191216173241622,지니,eHBM셀파수학교실-학습계통도(중등 5종),HB,1,750,신용카드,2019-12-16 17:33:43,구매확정,NaN,2019,12,4


In [51]:
# 결측치 처리
# '-', ' + 포'는 결측치로 생각
option = option.drop(option[option['결제방법'] == '-'].index)
option = option.drop(option[option['결제방법'] == ' + 포'].index)
option = option.dropna(subset=['결제방법'])

In [52]:
option['결제방법'].unique()

array(['신용카드', '가상계좌', '신 + 포', '포인트', '신 + 적 + 포', '신 + 적', '적 + 포',
       '웰컴마일', '신 + 웰컴', '적립금', '후불', '후 + 포', '후 + 적 + 포', '후 + 적',
       '정 + 포', '정기결제', '정 + 적 + 포', '정 + 적', '현금간편결제', '현 + 적', '현 + 포',
       '현 + 적 + 포'], dtype=object)

In [53]:
# 내용을 문자열로 변환
option['결제방법'] = option['결제방법'].astype(str)

# 결제 방법이 여러 개인 경우 맨 앞의 결제방법으로 변환
option.loc[option.query('결제방법.str.startswith("신")').index, '결제방법'] = '신용카드'
option.loc[option.query('결제방법.str.startswith("적")').index, '결제방법'] = '적립금'
option.loc[option.query('결제방법.str.startswith("후")').index, '결제방법'] = '후불'
option.loc[option.query('결제방법.str.startswith("정")').index, '결제방법'] = '정기결제'
option.loc[option.query('결제방법.str.startswith("현")').index, '결제방법'] = '현금간편결제'

In [54]:
# 결제 방법별 판매 금액의 합계
option_sales_sum = option.groupby('결제방법')['판매금액'].sum().to_frame()
option_sales_sum['판매금액'].apply(lambda x: '{:,}원'.format(round(x))).to_frame()

,판매금액
결제방법,
가상계좌,"339,990원"
신용카드,"4,687,854,783원"
웰컴마일,"92,798,695원"
적립금,"43,975,975원"
정기결제,"1,256,518,000원"
포인트,"486,920,060원"
현금간편결제,"208,048,535원"
후불,"377,088,000원"


In [55]:
fig = px.pie(option_sales_sum, values='판매금액', names=option_sales_sum.index,
            title='결제 방법별 판매금액 합계')
fig.show()

In [56]:
# 결제 방법별 판매 금액의 평균
option_sales_mean = option.groupby('결제방법')['판매금액'].mean().to_frame()
option_sales_mean['판매금액'].apply(lambda x: '{:,}원'.format(round(x))).to_frame()

,판매금액
결제방법,
가상계좌,"17,000원"
신용카드,"40,346원"
웰컴마일,"14,646원"
적립금,"11,044원"
정기결제,"954,801원"
포인트,"19,515원"
현금간편결제,"41,585원"
후불,"1,376,234원"


In [57]:
fig = px.pie(option_sales_mean, values='판매금액', names=option_sales_mean.index,
            title='결제 방법별 판매금액 평균')
fig.show()